<a href="https://colab.research.google.com/github/syedmuneeb321/lanchain-practise/blob/main/DocChainMaster/3_StuffDocumentsChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
%pip install langchain python-dotenv ipykernel langchain-community pypdf bs4 arxiv pymupdf wikipedia langchain-text-splitters langchain-openai chromadb sentence_transformers langchain_huggingface faiss-cpu langchain_chroma duckdb pandas openai langchain-groq duckduckgo_search==5.3.1b1 mysql-connector-python SQLAlchemy validators==0.28.1 youtube_transcript_api unstructured pytube numexpr huggingface_hub

In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ['HF_TOKEN'] = userdata.get("HF_TOKEN")

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.0,


)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
docs = PyPDFLoader("/content/apjspeech.pdf").load()

In [ ]:
from langchain_core.prompts import PromptTemplate
template = """
write a concise and short summary of the following speech,
Speech: {text}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
)



In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

In [ ]:
output = chain.invoke(docs)

# According to latest docs

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List





template_2 = """
write a concise and short summary of the following speech,
Speech: {context}
"""

prompt_2 = PromptTemplate(
    template=template_2,
    input_variables=["context"],

)
chain = create_stuff_documents_chain(llm, prompt=prompt_2)

In [ ]:
chain.invoke({"context":docs})

In [ ]:
# prompt: now my use readme heading case is load ducemnt then llm create schmea

# from langchain.chains import create_extraction_chain

# schema = {
#     "properties": {
#         "summary": {"type": "string"},
#         "key_takeaways": {"type": "array", "items": {"type": "string"}},
#     },
#     "required": ["summary", "key_takeaways"],
# }

# chain = create_extraction_chain(schema, llm)
# output_3 = chain.invoke(docs)
# output_3

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List

class Summary(BaseModel):
    summary: str = Field(description="summary of the speech")





template_2 = """
Summarize the following content  in below format:
{format_instructions}


Content: {context}
"""

prompt_2 = PromptTemplate(
    template=template_2,
    input_variables=["context"],
    partial_variables={"format_instructions": JsonOutputParser(pydantic_object=Summary).get_format_instructions()},


)
chain = create_stuff_documents_chain(llm, prompt=prompt_2,output_parser=JsonOutputParser(pydantic_object=Summary))

In [ ]:
chain.invoke({"context":docs})

{'summary': "A P J Abdul Kalam's departing speech as President of India, highlighting his experiences and messages for the nation's development, including accelerating development, empowering villages, mobilizing rural core competence, and overcoming problems through partnership."}

# Shop Genie Document Schema Map

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List,Optional
class SmartphoneReview(BaseModel):
    """A review of a smartphone."""
    title: str = Field(..., description="The title of the smartphone review")
    url: Optional[str] = Field(None, description="The URL of the smartphone review")
    content: Optional[str] = Field(None, description="The main content of the smartphone review")
    pros: Optional[List[str]] = Field(None, description="The pros of the smartphone")
    cons: Optional[List[str]] = Field(None, description="The cons of the smartphone")
    highlights: Optional[dict] = Field(None, description="The highlights of the smartphone")
    score: Optional[float] = Field(None, description="The score of the smartphone")
class ListOfSmartphoneReviews(BaseModel):
    """A list of smartphone reviews."""
    reviews: List[SmartphoneReview] = Field(..., description="List of individual smartphone reviews")

In [ ]:
 prompt_template = """
You are a professional assistant tasked with extracting structured information from a blogs.

### Instructions:

1. **Product Details**: For each product mentioned in the blog post, populate the `products` array with structured data for each item, including:
   - `title`: The product name.
   - `url`: Link to the blog post or relevant page.
   - `content`: A concise summary of the product's main features or purpose.
   - `pros`: A list of positive aspects or advantages of the product.if available other wise extract blog content.
   - `cons`: A list of negative aspects or disadvantages.if available other wise extract blog content.
   - `highlights`: A dictionary containing notable features or specifications.if available other wise extract blog content.
   - `score`: A numerical rating score if available; otherwise, use `0.0`.

### Blogs Contents: {context}

After extracting all information, just return the response in the JSON structure given below. Do not add any extracted information. The JSON should be in a valid structure with no extra characters inside, like Python’s \n.


"""

In [ ]:
parser = JsonOutputParser(pydantic_object=ListOfSmartphoneReviews)



# Format the prompt with the full blogs content
blog_prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
blog_chain = create_stuff_documents_chain(llm, prompt=blog_prompt,output_parser=parser)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
page_url="https://www.nextpit.com/best-smartphones-under-1000"
blog_docs = WebBaseLoader(web_paths=[page_url], bs_get_text_kwargs={"separator": " ", "strip": True}).load()

In [ ]:
blog_chain.invoke({"context":blog_docs})

{'products': [{'title': 'Samsung Galaxy S24',
   'url': 'https://www.nextpit.com/samsung-galaxy-s24-review',
   'content': 'The Samsung Galaxy S24 is a high-end smartphone with a 6.2-inch display, powerful performance, and up to seven years of updates.',
   'pros': ['Powerful AI functions',
    'Outstanding display',
    'Compact and good feel',
    'Commendable update policy'],
   'cons': ['No camera upgrade',
    '128 GB UFS 3.1 memory',
    'Larger battery, shorter runtime',
    'Charging not up to date'],
   'highlights': {'processor': 'Snapdragon 8 Gen 3 (US) / Exynos 2400 (global)',
    'RAM': '8 GB LPDDR5X',
    'storage': '128 GB UFS 3.1',
    'camera': 'Wide: 50 MP, f/1.8, OIS / Ultra-wide: 12 MP, f/2.2 / 3x telephoto: 10 MP, f/2.4, OIS / Selfie: 12 MP, f/2.2'},
   'score': 0.0},
  {'title': 'Apple iPhone 16',
   'url': 'https://www.nextpit.com/apple-iphone-16-review',
   'content': 'The Apple iPhone 16 is a high-end smartphone with a streamlined selection of phones, expanded 